# Preprocessing: LL16


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data.

Really weird vertical lines in all channels after bipolar rereferencing, can't see the artifgacts in the raw data, but too many of them to use the data. MNE breaks trying to load them


In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'LL16'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/LL16/ieeg/LL16_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 940899 =      0.000 ...  1837.693 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/LL16/ieeg/LL16_raw_clean_ieeg.fif...
    Range : 0 ... 940899 =      0.000 ...  1837.693 secs
Ready.


In [4]:
## Channels did not include probe prefixes in bci data file, convert to descriptive names here

# read in csv with correct names
channel_csv = pd.read_csv(f"{preproc_data_dir}/../LL16-ChannelLabels.csv")

# convert to dict
channel_dict = dict(zip(channel_csv.bci_names, channel_csv.use_names))

# rename
mne.rename_channels(filtered_clean_fif.info, channel_dict)
mne.rename_channels(raw_clean_fif.info, channel_dict)

## Bipolar Rereferencing

In [5]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [6]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

LA1 LA2
LA2 LA3
LA3 LA4
LA4 LA5
LA5 LA6
LA6 LA7
LA7 LA8
LH1 LH2
LH2 LH3
LH3 LH4
LH4 LH5
LH5 LH6
LH6 LH7
LH7 LH8
LHP1 LHP2
LHP2 LHP3
LHP3 LHP4
LHP4 LHP5
LHP5 LHP6
LHP6 LHP7
LHP7 LHP8
LOF1 LOF2
LOF2 LOF3
LOF3 LOF4
LOF4 LOF5
LOF5 LOF6
LOF6 LOF7
LOF7 LOF8
LOF8 LOF9
LOF9 LOF11
LOF10 noref
LOF11 LOF12
LAC1 LAC2
LAC2 LAC3
LAC3 LAC4
LAC4 LAC5
LAC5 LAC6
LAC6 LAC7
LAC7 LAC8
LAC8 LAC9
LAC10 noref
LMC1 LMC2
LMC2 LMC3
LMC3 LMC4
LMC4 LMC5
LMC5 LMC6
LMC6 LMC7
LMC7 LMC8
LMC8 LMC9
LMC9 LMC10
ROF1 ROF2
ROF2 ROF3
ROF3 ROF4
ROF4 ROF5
ROF5 ROF6
ROF6 ROF7
ROF7 ROF8
ROF8 ROF9
ROF9 ROF10
ROF10 ROF11
ROF12 noref
RAC1 RAC2
RAC2 RAC3
RAC3 RAC4
RAC4 RAC5
RAC5 RAC6
RAC6 RAC7
RAC7 RAC8
RAC8 RAC9
RAC9 RAC10
RTL1 RTL2
RTL2 RTL3
RTL3 RTL4
RTL4 RTL5
RTL5 RTL6
RTL6 RTL7
RTL7 RTL8
EKG noref
CH87 noref
CH88 noref
CH89 noref
CH90 noref
CH91 noref
CH92 noref
CH93 noref
CH94 noref
CH95 noref
CH96 noref
CH97 noref
CH98 noref
CH99 noref
CH100 noref
CH101 noref
CH102 noref
CH103 noref
CH104 noref
CH105 noref
CH106 noref
CH107 n

In [7]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 940899  =      0.000 ...  1837.693 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=74, n_times=940900
    Range : 0 ... 940899 =      0.000 ...  1837.693 secs
Ready.


/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/mne/io/reference.py:557: RuntimeWarning: invalid value encountered in matmul
  ref_data = multiplier @ inst._data


Added the following bipolar channels:
LA1-LA2, LA2-LA3, LA3-LA4, LA4-LA5, LA5-LA6, LA6-LA7, LA7-LA8, LH1-LH2, LH2-LH3, LH3-LH4, LH4-LH5, LH5-LH6, LH6-LH7, LH7-LH8, LHP1-LHP2, LHP2-LHP3, LHP3-LHP4, LHP4-LHP5, LHP5-LHP6, LHP6-LHP7, LHP7-LHP8, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF11, LOF11-LOF12, LAC1-LAC2, LAC2-LAC3, LAC3-LAC4, LAC4-LAC5, LAC5-LAC6, LAC6-LAC7, LAC7-LAC8, LAC8-LAC9, LMC1-LMC2, LMC2-LMC3, LMC3-LMC4, LMC4-LMC5, LMC5-LMC6, LMC6-LMC7, LMC7-LMC8, LMC8-LMC9, LMC9-LMC10, ROF1-ROF2, ROF2-ROF3, ROF3-ROF4, ROF4-ROF5, ROF5-ROF6, ROF6-ROF7, ROF7-ROF8, ROF8-ROF9, ROF9-ROF10, ROF10-ROF11, RAC1-RAC2, RAC2-RAC3, RAC3-RAC4, RAC4-RAC5, RAC5-RAC6, RAC6-RAC7, RAC7-RAC8, RAC8-RAC9, RAC9-RAC10, RTL1-RTL2, RTL2-RTL3, RTL3-RTL4, RTL4-RTL5, RTL5-RTL6, RTL6-RTL7, RTL7-RTL8
Writing /home/brooke/pacman/preprocessing/LL16/ieeg/LL16_bp_filtered_clean_data.fif


/tmp/ipykernel_47203/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL16/ieeg/LL16_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL16/ieeg/LL16_bp_filtered_clean_data.fif
[done]
Reading 0 ... 940899  =      0.000 ...  1837.693 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=74, n_times=940900
    Range : 0 ... 940899 =      0.000 ...  1837.693 secs
Ready.
Added the following bipolar channels:
LA1-LA2, LA2-LA3, LA3-LA4, LA4-LA5, LA5-LA6, LA6-LA7, LA7-LA8, LH1-LH2, LH2-LH3, LH3-LH4, LH4-LH5, LH5-LH6, LH6-LH7, LH7-LH8, LHP1-LHP2, LHP2-LHP3, LHP3-LHP4, LHP4-LHP5, LHP5-LHP6, LHP6-LHP7, LHP7-LHP8, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF11, LOF11-LOF12, LAC1-LAC2, LAC2-LAC3, LAC3-LAC4, LAC4-LAC5, LAC5-LAC6, LAC6-LAC7, LAC7-LAC8, LAC8-LAC9, LMC1-LMC2, LMC2-LMC3, LMC3-LMC4, LMC4-LMC5, LMC5-LMC6, LMC6-LMC7, LMC7-LMC8, LMC8-LMC9, LMC9-LMC10, ROF1-ROF2, ROF2-ROF3, ROF3-ROF4, ROF4-ROF5, ROF5-ROF6, ROF6-ROF7, ROF7-ROF8, ROF8-ROF9, ROF9-ROF10, ROF10-ROF11, RAC1-RAC2, RAC2-RAC3

/tmp/ipykernel_47203/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL16/ieeg/LL16_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL16/ieeg/LL16_bp_raw_clean_data.fif
[done]


In [4]:
bad_annotation = mne.Annotations(onset = [301], duration =[1], description=['BAD'])



In [3]:
# Visualize it #
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')
bp_filtered_fif.annotations.append([301], [1], ['BAD'])
bp_filtered_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 70, clipping = None, event_color = 'r')

Opening raw data file /home/brooke/pacman/preprocessing/LL16/ieeg/LL16_bp_filtered_clean_data.fif...
    Range : 0 ... 940899 =      0.000 ...  1837.693 secs
Ready.
480 events found
Event IDs: [0 1]


/tmp/ipykernel_52640/1245655609.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL16/ieeg/LL16_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


Using qt as 2D backend.


/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/numpy/core/_methods.py:118: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/numpy/core/_methods.py:152: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/scipy/stats/_stats_py.py:3146: RuntimeWarning: invalid value encountered in subtract
  z = (scores - mn) / std
/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, 

Channels marked as bad:
['CH100', 'CH101', 'CH102', 'CH103', 'CH104', 'CH105', 'CH106', 'CH107', 'CH108', 'CH109', 'CH110', 'CH111', 'CH112', 'CH113', 'CH114', 'CH115', 'CH116', 'CH117', 'CH118', 'CH119', 'CH120', 'CH121', 'CH122', 'CH123', 'CH124', 'CH125', 'CH126', 'CH127', 'CH128', 'CH129', 'CH130', 'CH131', 'CH132', 'CH133', 'CH134', 'CH135', 'CH136', 'CH137', 'CH138', 'CH139', 'CH140', 'CH141', 'CH142', 'CH143', 'CH144', 'CH145', 'CH146', 'CH147', 'CH87', 'CH88', 'CH89', 'CH90', 'CH91', 'CH92', 'CH93', 'CH94', 'CH95', 'CH96', 'CH97', 'CH98', 'CH99', 'EKG', 'LAC10', 'LOF10', 'ROF12']


In [4]:
# From interactive sessions
interactive_annot = bp_filtered_fif.annotations
for x in range(0, len(bp_filtered_fif.annotations)):
    print(interactive_annot[x])
    
    
bp_filtered_fif.annotations.save('../ieeg/saved_annotations2.fif', overwrite = True)    

OrderedDict([('onset', 64.159378), ('duration', 0.32196807861328125), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 192.390488), ('duration', 1.2581024169921875), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 301.0), ('duration', 1.0), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 303.383575), ('duration', 0.651885986328125), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 349.139984), ('duration', 0.594024658203125), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 430.691406), ('duration', 2.90625), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 496.462891), ('duration', 0.775390625), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 537.207031), ('duration', 2.39453125), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 557.249254699417), ('duration', 0.5468742898179926), ('description', 'BAD'), ('orig_time', None)]

/tmp/ipykernel_52640/1164348305.py:7: RuntimeWarning: This filename (../ieeg/saved_annotations2.fif) does not conform to MNE naming conventions. All annotations files should end with -annot.fif, -annot.fif.gz, _annot.fif, _annot.fif.gz, .txt or .csv
  bp_filtered_fif.annotations.save('../ieeg/saved_annotations2.fif', overwrite = True)


## Epoching the data 

### Onset

In [ ]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

### Trial End

In [ ]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

### Last Away

In [ ]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

In [ ]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [ ]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


### First Dot

In [ ]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

In [ ]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [ ]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


## First Move

In [ ]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

In [ ]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [ ]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


## Ghost Attack

In [ ]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

In [ ]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [ ]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
